In [24]:
from torchvision import datasets
from torchvision import transforms

from torch.utils.tensorboard import SummaryWriter

In [25]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [26]:
train_data = datasets.CIFAR10(
    root = 'data',
    train = True,
    transform = transform,
    download = True
)

In [27]:
test_data = datasets.CIFAR10(
    root = 'data',
    train = False,
    transform = transform,
    download = True
)

In [28]:
train_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [29]:
test_data

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [30]:
train_data.data.shape

(50000, 32, 32, 3)

In [31]:
test_data.data.shape

(10000, 32, 32, 3)

In [32]:
from torch.utils.data import DataLoader

batch_size = 128

loaders = {
    'train': DataLoader(train_data,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=0,
                        drop_last=True),
    'test': DataLoader(test_data,
                       batch_size=batch_size,
                       shuffle=True,
                       num_workers=0)
}

In [33]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x2d23d692d50>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x2d23d693110>}

In [34]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(500, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 500)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [35]:
import os
import torch
device = torch.device('cuda:0')
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 降低学习率
criterion = nn.CrossEntropyLoss()

os.makedirs('log', exist_ok=True)
writer = SummaryWriter('log')

def train_and_test(epochs):
    for epoch in range(epochs):
        model.train()
        train_avg_loss = 0

        for batch_idx, (data, target) in enumerate(loaders['train']):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_avg_loss += loss.item() * data.size(0)
            if batch_idx % 20 == 0:
                print(f"Train Epoch: {epoch} [{batch_idx * len(data)} / {len(loaders['train'].dataset)}]")

        train_avg_loss /= len(loaders['train'].dataset)

        model.eval()
        test_loss = 0
        correct = 0

        with torch.no_grad():
            for data, target in loaders['test']:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item() * data.size(0)  # 累加总损失
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(loaders['test'].dataset)  # 除以总样本数得到平均损失
        accuracy = 100. * correct / len(loaders['test'].dataset)
        print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders['test'].dataset)} ({accuracy:.2f}%)")

        writer.add_scalars('loss/', {'train': train_avg_loss, 'val': test_loss}, epoch)

In [36]:
train_and_test(11)

Train Epoch: 0 [0 / 50000]
Train Epoch: 0 [2560 / 50000]
Train Epoch: 0 [5120 / 50000]
Train Epoch: 0 [7680 / 50000]
Train Epoch: 0 [10240 / 50000]
Train Epoch: 0 [12800 / 50000]
Train Epoch: 0 [15360 / 50000]
Train Epoch: 0 [17920 / 50000]
Train Epoch: 0 [20480 / 50000]
Train Epoch: 0 [23040 / 50000]
Train Epoch: 0 [25600 / 50000]
Train Epoch: 0 [28160 / 50000]
Train Epoch: 0 [30720 / 50000]
Train Epoch: 0 [33280 / 50000]
Train Epoch: 0 [35840 / 50000]
Train Epoch: 0 [38400 / 50000]
Train Epoch: 0 [40960 / 50000]
Train Epoch: 0 [43520 / 50000]
Train Epoch: 0 [46080 / 50000]
Train Epoch: 0 [48640 / 50000]

Test set: Average loss: 1.6723, Accuracy: 4083/10000 (40.83%)
Train Epoch: 1 [0 / 50000]
Train Epoch: 1 [2560 / 50000]
Train Epoch: 1 [5120 / 50000]
Train Epoch: 1 [7680 / 50000]
Train Epoch: 1 [10240 / 50000]
Train Epoch: 1 [12800 / 50000]
Train Epoch: 1 [15360 / 50000]
Train Epoch: 1 [17920 / 50000]
Train Epoch: 1 [20480 / 50000]
Train Epoch: 1 [23040 / 50000]
Train Epoch: 1 [25600